In [3]:
import numpy as np
import pandas as pd
import pickle
import oenotebook as oenb
from openeye.oechem import *
from openmoltools.openeye import *
from openeye.oeiupac import *

#from extraction import main
from test_extract import main

### Part 1: Checking SD Tags of Orion Output Data

In [4]:
#Use output .oeb file and the following lines to create OEMol object with SD Tags
input_file = 'output/results/IDAC_solvation_success.oeb'
its = oechem.oemolistream(input_file)
molecule = oechem.OEMol()
oechem.OEReadMolecule(its, molecule)

tag_name = 'name'
#Check if a Certain SD Tag exists and its value
oechem.OEGetSDData(molecule, tag_name)

'formic acid'

In [5]:
def DumpSDData(mol):
    print("SD data of {}:".format(mol.GetTitle()))
    #Loop over SD Data
    for dp in oechem.OEGetSDDataPairs(mol):
        print(dp.GetTag())

In [6]:
##Use imported function to convert .oeb input into a .csv
# main('output/results/IDAC_solvation_success.oeb', 'output/results/IDAC_solvation_success.csv')
# main('output/results/IDAC_solvation_fail.oeb', 'output/results/IDAC_solvation_fail.csv')
# main('output/results/IDAC_ssolv_success.oeb', 'output/results/IDAC_ssolv_success.csv')
# main('output/results/IDAC_ssolv_fail.oeb', 'output/results/IDAC_ssolv_fail.csv')

In [7]:
def counting(table):
    i = 0
    for key in table:
        i += len(table[key])
    return i

In [8]:
#Read in output files from Orion
solv_success = pd.read_csv('output/results/IDAC_solvation_success.csv')
solv_fail = pd.read_csv('output/results/IDAC_solvation_fail.csv')

#Remove leading whitespace in column names
solv_success.columns = [x.lstrip() for x in list(solv_success.columns)]
solv_fail.columns = [x.lstrip() for x in list(solv_fail.columns)]

ssolv_success = pd.read_csv('output/results/IDAC_ssolv_success.csv')
ssolv_fail = pd.read_csv('output/results/IDAC_ssolv_fail.csv')

#Remove leading whitespace in column names
ssolv_success.columns = [x.lstrip() for x in list(ssolv_success.columns)]
ssolv_fail.columns = [x.lstrip() for x in list(ssolv_fail.columns)]

In [9]:
#Combined list of solvation and self-solvation calculation outputs
combined_solv = solv_fail.append(solv_success)
combined_ssolv = ssolv_fail.append(ssolv_success)

In [10]:
#Import pickled dictionary to assess which entries are missing
temp_dict = pickle.load(open('temp_dict.p', 'rb'))
ref_table = pickle.load(open('input/final_table.p', 'rb'))
solv_table = pickle.load(open('input/final_solv.pkl', 'rb'))
ssolv_table = pickle.load(open('input/final_ssolv.pkl', 'rb'))

In [11]:
#Create dictionaries with all compounds and the temperatures they were calculated at
solv_dict = {}
for name in combined_solv.Molecule.unique():
    subset = combined_solv[combined_solv.Molecule == name]
    solv_dict[name] = [x for x in subset['Temperature(K)']]

ssolv_dict = {}
for name in combined_ssolv.Molecule.unique():
    subset = combined_ssolv[combined_ssolv.Molecule == name]
    ssolv_dict[name] = [x for x in subset['Temperature(K)']]

In [12]:
#Convert compound names to smiles to compare between dictionaries 
smiles = {}
for name in temp_dict:
    mol_from_name = OEMol()
    OEParseIUPACName(mol_from_name, name)
    smiles[name] = OECreateCanSmiString(mol_from_name)

smiles_solv = {}
for name in solv_dict:
    mol_from_name = OEMol()
    OEParseIUPACName(mol_from_name, name)
    smiles_solv[name] = OECreateCanSmiString(mol_from_name)
    
smiles_ssolv = {}
for name in ssolv_dict:
    mol_from_name = OEMol()
    OEParseIUPACName(mol_from_name, name)
    smiles_ssolv[name] = OECreateCanSmiString(mol_from_name)

In [13]:
#Create dictionaries with keys as SMILES strings for comparison
total_dict = {}
for name in temp_dict:
    total_dict[smiles[name]] = temp_dict[name]
    
new_solv_dict = {}
for name in solv_dict:
    new_solv_dict[smiles_solv[name]] = solv_dict[name]

new_ssolv_dict = {}
for name in ssolv_dict:
    new_ssolv_dict[smiles_ssolv[name]] = ssolv_dict[name]

In [14]:
print('Temp_dict:', counting(total_dict))
print('Solvation_dict:', counting(new_solv_dict))
print('Self-Solvation_dict:', counting(new_ssolv_dict))

Temp_dict: 240
Solvation_dict: 237
Self-Solvation_dict: 213


### Searching for Dropped Values within Keys

In [15]:
dropped_solv = {}
for key in new_solv_dict:
    if sorted(total_dict[key]) != sorted(new_solv_dict[key]):
        dropped_solv[key] = list(set(total_dict[key]) ^ set(new_solv_dict[key]))
        
dropped_ssolv = {}        
for key in new_ssolv_dict:
    if sorted(total_dict[key]) != sorted(new_ssolv_dict[key]):
        dropped_ssolv[key] = list(set(total_dict[key]) ^ set(new_ssolv_dict[key]))

In [16]:
print(counting(dropped_solv))
print(counting(dropped_ssolv))

1
10


### Searching for Dropped Keys

In [17]:
original_smiles = set([key for key in total_dict])
solv_smiles = set([key for key in new_solv_dict])
ssolv_smiles = set([key for key in new_ssolv_dict])

solv_difference = list(original_smiles ^ solv_smiles)
ssolv_difference = list(original_smiles ^ ssolv_smiles)

bob = []
for smile in solv_difference:
    dropped_solv[smile] = total_dict[smile]

for smile in ssolv_difference:
    dropped_ssolv[smile] = total_dict[smile]

In [18]:
print(counting(dropped_solv))
print(counting(dropped_ssolv))

3
27


### Printing Dropped Calculations

In [19]:
solv_rip = pd.DataFrame()
for key in dropped_solv:
    subset = solv_table[solv_table['solute_SMILES'] == key]
    for temp in dropped_solv[key]:
        solv_rip = solv_rip.append(subset[subset.temperature == temp])

In [20]:
ssolv_rip = pd.DataFrame()
for key in dropped_ssolv:
    subset = ssolv_table[ssolv_table.solvents == key]
    for temp in dropped_ssolv[key]:
        ssolv_rip = ssolv_rip.append(subset[subset.temperature == temp])

In [21]:
pd.DataFrame.to_pickle(solv_rip, 'error/dropped_solv.pkl')
pd.DataFrame.to_pickle(ssolv_rip, 'error/dropped_ssolv.pkl')

### Adjusting Size of Solvation and Self-Solvation Sets

In [22]:
solv_success.head()
len(solv_success)

184

In [23]:
ssolv_success.head()
len(ssolv_success)

175

In [24]:
#Generate lists of all compounds and the calculated temperatures for solv and self-solv
solv_yes = []
for name in solv_success.Molecule.unique():
    subset = solv_success[solv_success.Molecule == name]
    for temp in subset['Temperature(K)']:
        solv_yes.append(str(name) + ';' + str(temp))
        
ssolv_yes = []
for name in ssolv_success.Molecule.unique():
    subset = ssolv_success[ssolv_success.Molecule == name]
    for temp in subset['Temperature(K)']:
        ssolv_yes.append(str(name) + ';' + str(temp))

In [25]:
A = set(solv_yes)
B = set(ssolv_yes)

diff = A^B
print(len(diff))
print(len(A-diff))
print(len(B-diff))

79
140
140


### Remove the Calculations without Both Solvation and Self-Solvation

In [26]:
adjusted_solv = list(A-diff)
adjusted_ssolv = list(B-diff)

In [27]:
new_solv = pd.DataFrame()
for entry in adjusted_solv:
    subset = solv_success[solv_success.Molecule == entry.split(';')[0]]
    new_solv = new_solv.append(subset[subset['Temperature(K)'] == float(entry.split(';')[1])])

In [28]:
print(len(new_solv))
new_solv.head()

140


,Molecule,Temperature(K),Pressure(atm),Solvents(smiles),Molar_fractions,FE(kcal/mol),DFE(kcal/mol),Density_Start(g/ml),DDensity_Start(g/ml),Density_Final(g/ml),DDensity_Final(g/ml),IDAC_expt
141,o-xylene,308.15,1.0,[H]O[H],1.0,-1.026401,0.088860,0.977928,0.010226,0.969540,0.010518,29850.000
63,tetrahydrofuran,338.05,1.0,[H]O[H],1.0,-1.465516,0.073407,0.952899,0.011317,0.935907,0.015147,9.400
171,hexan-2-one,298.15,1.0,[H]O[H],1.0,-3.177169,0.095033,0.992495,0.008310,0.980627,0.009097,355.700
17,2-[2-(2-hydroxyethoxy)ethoxy]ethanol,378.15,1.0,[H]O[H],1.0,-6.448490,0.272052,0.898001,0.011821,0.895903,0.016699,0.807
12,butan-1-amine,293.15,1.0,[H]O[H],1.0,-2.419833,0.087518,0.992998,0.011627,0.987103,0.010507,4.000


In [29]:
new_ssolv = pd.DataFrame()
for entry in adjusted_ssolv:
    subset = ssolv_success[ssolv_success.Molecule == entry.split(';')[0]]
    new_ssolv = new_ssolv.append(subset[subset['Temperature(K)'] == float(entry.split(';')[1])])
    
new_ssolv['Solvents(smiles)'] = [x.lstrip() for x in new_ssolv['Solvents(smiles)']]

In [30]:
print(len(new_ssolv))
new_ssolv.head()

140


,Molecule,Temperature(K),Pressure(atm),Solvents(smiles),Molar_fractions,FE(kcal/mol),DFE(kcal/mol),Density_Start(g/ml),DDensity_Start(g/ml),Density_Final(g/ml),DDensity_Final(g/ml),IDAC_expt
135,o-xylene,308.15,1.0,Cc1ccccc1C,1.0,-5.693364,0.104985,0.837511,0.010715,0.831558,0.009716,29850.000
76,tetrahydrofuran,338.05,1.0,C1CCOC1,1.0,-3.910854,0.104257,0.851238,0.011389,0.842281,0.013976,9.400
56,hexan-2-one,298.15,1.0,CCCCC(=O)C,1.0,-6.264420,0.138696,0.790474,0.011176,0.785346,0.010374,355.700
60,2-[2-(2-hydroxyethoxy)ethoxy]ethanol,378.15,1.0,C(COCCOCCO)O,1.0,-7.390926,0.357555,1.075257,0.012691,1.069301,0.012923,0.807
173,butan-1-amine,293.15,1.0,CCCCN,1.0,-3.446458,0.127553,0.760285,0.008548,0.759044,0.010394,4.000


In [31]:
ref_table.head()

,Solute,SMILES,Temp,Measured,Uncertain,Method,Ref.,Notes
923,"1,2,4-trimethylbenzene",Cc1ccc(c(c1)C)C,288.15,127600.0,0.0,NaN,7,NaN
924,"1,2,4-trimethylbenzene",Cc1ccc(c(c1)C)C,298.15,115000.0,0.0,NaN,"9,12",NaN
926,"1,2,4-trimethylbenzene",Cc1ccc(c(c1)C)C,308.15,107400.0,0.0,NaN,7,NaN
927,"1,2,4-trimethylbenzene",Cc1ccc(c(c1)C)C,318.15,96340.0,0.0,NaN,7,NaN
487,"1,2-butanediol",CCC(CO)O,299.15,2.0,0.2,DP,62,NaN


In [32]:
##Re-insert experimental errors from IDAC paper
error_exp = []
for smile,temp in zip(new_ssolv['Solvents(smiles)'], new_ssolv['Temperature(K)']):
    subset = ref_table[ref_table['SMILES'] == smile]
    subset = subset[subset['Temp'] == temp]
    error_exp.append(float(subset['Uncertain'].item()))

In [33]:
new_ssolv['Uncertain_exp'] = error_exp
uncertain = new_ssolv['Uncertain_exp']
uncertain.unique()

array([ 0.  ,  0.1 , 30.  ,  0.3 ,  0.2 ,  0.5 , 25.  ,  0.09,  0.03])

In [34]:
new_ssolv[new_ssolv['Uncertain_exp'] == 30.]

,Molecule,Temperature(K),Pressure(atm),Solvents(smiles),Molar_fractions,FE(kcal/mol),DFE(kcal/mol),Density_Start(g/ml),DDensity_Start(g/ml),Density_Final(g/ml),DDensity_Final(g/ml),IDAC_expt,Uncertain_exp
116,"1,3-dichloroprop-1-ene",293.15,1.0,C(C=CCl)Cl,1.0,-4.573972,0.087441,1.156206,0.018709,1.143907,0.02212,1360.0,30.0


In [35]:
ratios = []
len(new_ssolv[new_ssolv['Uncertain_exp'] == 0.])
for index, compound in enumerate(new_ssolv['Molecule']):
    subset = new_ssolv[new_ssolv['Molecule'] == compound]
    for i in range(len(subset)):
        if list(subset['Uncertain_exp'])[i] == 0:
            ratios.append('0' + ';' + str(compound))
        else:
            ratios.append(str(list(subset['Uncertain_exp'])[i]/list(subset['IDAC_expt'])[i]) + ';' + str(compound))

### Part 2: Calculating IDAC from Orion Output Data

Using the following formula: $$\gamma_{i}^{\infty} = exp\bigg(\frac{\Delta G_{i}^{solv} - \Delta G_{i}^{self-solv}}{k_{B}T} \bigg) \cdot \frac{\rho_{molar}^{solvent}}{\rho_{molar}^{pure-solute}} $$

In [37]:
def calc_idac(temperature, G_solvation, G_self_solvation, top_density, bottom_density):
    '''
    top_density: final density in solvation
    bottom_density: final density in self-solvation
    '''
    IDAC_calc = []
    k = 0.0019872041 #units in kcal/mol*K
    for i in range(len(temperature)):
        value = np.exp((G_solvation[i] - G_self_solvation[i])/(k*temperature[i])) * (top_density[i]/bottom_density[i])
        IDAC_calc.append('{:.2f}'.format(value))
    return IDAC_calc

In [38]:
def exp_error(temperature, IDAC_exp, error):
    error_exp = []
    k = 0.0019872041
    for i in range(len(temperature)):
        value = ((k*temperature[i])/IDAC_exp[i])**2 * error[i]**2
        error_exp.append('{:.5f}'.format(value))
    return error_exp

In [39]:
def calc_error(temperature, solv_error, ssolv_error, top_density, bottom_density, top_density_error, bottom_density_error):
    error_calc = []
    k = 0.0019872041
    for i in range(len(temperature)):
        value = solv_error[i]**2 + ssolv_error[i]**2 + ((k*temperature[i])/top_density[i])**2 * top_density_error[i]**2 + ((k*temperature[i])/bottom_density[i])**2 * bottom_density_error[i]**2
        error_calc.append('{:.2f}'.format(value))
    return error_calc

In [40]:
def calculate_IDAC(solvation, self_solvation):
    table = pd.DataFrame(columns = ['Name', 'Temp', 'IDAC_calc', 'IDAC_expt'])
    table['Name'] = solvation['Molecule']
    table['Temp'] = solvation['Temperature(K)']
    table['IDAC_calc'] = calc_idac([x for x in table['Temp']],
                                   [x for x in solvation['FE(kcal/mol)']],
                                   [x for x in self_solvation['FE(kcal/mol)']], 
                                   [x for x in solvation['Density_Final(g/ml)']],
                                   [x for x in self_solvation['Density_Final(g/ml)']])
    table['IDAC_expt'] = [str(x) for x in solvation['IDAC_expt']]
    table['kT*ln(IDAC_calc)_Error'] = calc_error([x for x in table['Temp']],
                                                 [x for x in solvation['DFE(kcal/mol)']],
                                                 [x for x in self_solvation['DFE(kcal/mol)']],
                                                 [x for x in solvation['Density_Final(g/ml)']],
                                                 [x for x in self_solvation['Density_Final(g/ml)']],
                                                 [x for x in solvation['DDensity_Final(g/ml)']],
                                                 [x for x in self_solvation['DDensity_Final(g/ml)']])
    
    table['kT*ln(IDAC_exp)_Error'] = exp_error([x for x in self_solvation['Temperature(K)']],
                                               [x for x in self_solvation['IDAC_expt']],
                                               [x for x in self_solvation['Uncertain_exp']])
    return table

In [41]:
new_solv.columns = [x.lstrip() for x in list(new_solv.columns)]
new_ssolv.columns = [x.lstrip() for x in list(new_ssolv.columns)]

In [42]:
final = calculate_IDAC(new_solv, new_ssolv)

In [43]:
final['kT*ln(IDAC_calc)_Error'] = [float(x) for x in final['kT*ln(IDAC_calc)_Error']]
final['kT*ln(IDAC_exp)_Error'] = [float(x) for x in final['kT*ln(IDAC_exp)_Error']]

In [44]:
final[final['IDAC_calc'] == '0.00'] 

,Name,Temp,IDAC_calc,IDAC_expt,kT*ln(IDAC_calc)_Error,kT*ln(IDAC_exp)_Error
159,tributyl phosphate,298.15,0.00,2.34,0.03,0.0


In [45]:
#Drop the entries with calculated IDAC values of 0
final = final[final['IDAC_calc'] != '0.00']
final = final.reset_index(drop = True)

In [46]:
final.sort_values(by = ['Name', 'Temp']).head()

,Name,Temp,IDAC_calc,IDAC_expt,kT*ln(IDAC_calc)_Error,kT*ln(IDAC_exp)_Error
33,"1,2,4-trimethylbenzene",288.15,9806.35,127600.0,0.02,0.0
65,"1,2,4-trimethylbenzene",298.15,10763.78,115000.0,0.02,0.0
138,"1,2,4-trimethylbenzene",318.15,10386.48,96340.0,0.02,0.0
131,"1,2-dichloroethylene",285.15,975.72,385.6,0.01,0.0
118,"1,2-dichloroethylene",293.15,1183.72,215.5,0.01,0.0


In [47]:
pd.DataFrame.to_pickle(final, 'output/IDAC_FINAL.pkl')

### old stuff

In [47]:
results = oenb.read_file_to_dataframe("output/results_final.oeb.gz")

In [48]:
smile = 'CCCCOC(=O)C'
sub_group = results[results['Solute SMILES'] == smile]

In [49]:
sub_group[sub_group['Solvent name'] == 'water']

,Molecule,DG_self_solv (kcal/mol),dDG_self_solv (kcal/mol),"expt IDAC, error",dDG_solv (kcal/mol),DG_solv (kcal/mol),"kT log(gamma)_calc, error (kcal/mol)",expt IDAC,Temperature (K),Solvent SMILES,kT log(gamma)_calc (kcal/mol),kT log(gamma)_expt (kcal/mol),Solute SMILES,"kT log(gamma)_expt, error (kcal/mol)",Solvent name
211,<oechem.OEMol; proxy of <Swig Object of type '...,-6.93106,0.19023,3,0.12814,-3.929219,0.222547,613.0,273.35,[H]O[H],2.913815,3.48414,CCCCOC(=O)C,0.004894,water


In [50]:
calc_idac = np.exp((2.913815)/(0.0019872041*273.35))

In [51]:
col = ['Name', 'Temperature (K)', '$\Delta G_{sol}$ (kcal/mol)', '$\Delta G_{selfsolv}$ (kcal/mol)', 'IDAC_calc', 'IDAC_exp']
compare = pd.DataFrame(columns = col)
compare.append(pd.DataFrame([['Chris', 298.15, -3.293, -6.239, 161.2, 814.00], ['Guilherme', 273.35, -3.929, -6.931, 213.6, 613]], columns = col))

,Name,Temperature (K),$\Delta G_{sol}$ (kcal/mol),$\Delta G_{selfsolv}$ (kcal/mol),IDAC_calc,IDAC_exp
0,Chris,298.15,-3.293,-6.239,161.2,814.0
1,Guilherme,273.35,-3.929,-6.931,213.6,613.0
